# Set up and global variables

In [ ]:
from pathlib import Path
from copy import deepcopy

import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from IPython.display import display, HTML
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.pipeline import Pipeline

from src.prioritization import *

In [ ]:
os.environ["CONFIG_ENV"] = "debug"
if False:
    os.environ["CONFIG_ENV"] = "production"

from config import load_config
config = load_config()

RESOLUTION = config['DEFAULTS']['resolution']
SEED = config['DEFAULTS']['random_seed']

# input data
BENCHMARK_PATH = config['PATHS']['benchmark_dataset']
STORAGE_PATH = config['PATHS']['storage']

# output data
IMAGE_DIR = config['PATHS']['images'] / 'modelling'
FINAL_MODEL_PATH = BENCHMARK_PATH / "final_teacher_model.pkl"
FINAL_FEATURES_PATH = BENCHMARK_PATH / "final_selected_features.pkl"



os.makedirs(IMAGE_DIR, exist_ok=True)

***

# Loading data

In [ ]:
items = pd.read_csv(STORAGE_PATH / 'items.csv', index_col=0)
defects = pd.read_csv(STORAGE_PATH / f'defects.csv', index_col=0)

df = pd.read_csv(BENCHMARK_PATH / 'benchmark_dataset.csv')

In [ ]:
BENCHMARK_PATH / 'benchmark_dataset.csv'

In [ ]:
left_discrete_features = [col for col in df.columns if col.endswith('(Left Discrete)')]
right_discrete_features = [col for col in df.columns if col.endswith('(Right Discrete)')]
left_continuous_features = [col for col in df.columns if col.endswith('(Left Continuous)')]
right_continuous_features = [col for col in df.columns if col.endswith('(Right Continuous)')]

if any(map(lambda x: len(x) == 0, [left_discrete_features, right_discrete_features, left_continuous_features, right_continuous_features])):
    raise ValueError('Some of the feature sets are empty')

***

# Feature engineering

In [ ]:
def remove_suffix(col: str) -> str:
    """Return base name before the first ' (' occurrence; fall back to original."""
    if not isinstance(col, str):
        return col
    idx = col.find(" (")
    return col[:idx] if idx != -1 else col

In [ ]:
left_discrete_values = df[left_discrete_features].rename(columns=remove_suffix)
right_discrete_values = df[right_discrete_features].rename(columns=remove_suffix)

left_continuous_values = df[left_continuous_features].rename(columns=remove_suffix)
right_continuous_values = df[right_continuous_features].rename(columns=remove_suffix)

## Difference features

In [ ]:
discrete_diff = left_discrete_values - right_discrete_values
discrete_diff = discrete_diff.add_suffix(' (Discrete Diff)')

continuous_diff = left_continuous_values - right_continuous_values
continuous_diff = continuous_diff.add_suffix(' (Continuous Diff)')

## Binary flags

In [ ]:
discrete_is_larger = left_discrete_values > right_discrete_values
discrete_is_larger = discrete_is_larger.add_suffix(' (Discrete >)')

continuous_is_larger = left_continuous_values > right_continuous_values
continuous_is_larger = continuous_is_larger.add_suffix(' (Continuous >)')

In [ ]:
left_is_extreme_max = left_discrete_values == 5
left_is_extreme_max = left_is_extreme_max.add_suffix(' (Left Max)')
left_is_extreme_min = left_discrete_values == 1
left_is_extreme_min = left_is_extreme_min.add_suffix(' (Left Min)')

## Item and defect metadata

In [ ]:
left_type = defects['defect type'].loc[df['left']].reset_index(drop=True).rename('left')
right_type = defects['defect type'].loc[df['right']].reset_index(drop=True).rename('right')

item_topic = items['topic'].loc[df['item']].reset_index(drop=True).rename('item')

In [ ]:
metadata_encoder = OneHotEncoder()

metadata = metadata_encoder.fit_transform(pd.concat([
    left_type,
    right_type,
    item_topic
], axis=1))

metadata = pd.DataFrame(metadata.toarray(), columns=metadata_encoder.get_feature_names_out())

# Feature groups and combined dataframe

In [ ]:
engineered_df = pd.concat([
    df[left_discrete_features],
    df[right_discrete_features],
    discrete_diff,
    df[left_continuous_features],
    df[right_continuous_features],
    continuous_diff,
    discrete_is_larger,
    continuous_is_larger,
    left_is_extreme_max,
    left_is_extreme_min,
    metadata,
], axis=1)

print("Final engineered dataframe shape:", engineered_df.shape)

In [ ]:
feature_groups = {
    "Left Discrete": left_discrete_features,
    "Right Discrete": right_discrete_features,
    "Discrete Diff": discrete_diff.columns.tolist(),
    "Left+Right Continuous": left_continuous_features + right_continuous_features,
    "Continuous Diff": continuous_diff.columns.tolist(),
    "Derived Rules": discrete_is_larger.columns.tolist() + continuous_is_larger.columns.tolist(),
    "Additional Rules": left_is_extreme_max.columns.tolist() + left_is_extreme_min.columns.tolist(),
    "Metadata": metadata.columns.tolist(),
    "All Features": engineered_df.columns.tolist()
}

***

# Training loop

In [ ]:
models = {
    "Random Forest": RandomForestClassifier(
        random_state=SEED,
        max_depth=3
    ),

    "Gradient Boosting": GradientBoostingClassifier(
        random_state=SEED,
        max_depth=3
    ),

    "Logistic Regression": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            random_state=SEED,
            max_iter=5000,
            solver="lbfgs"
        ))
    ]),
    "LASSO Logistic": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            random_state=SEED,
            max_iter=5000,
            solver="liblinear",
            penalty="l1"
        ))
    ])
}

In [ ]:
y = df['left won']

groups = df['submission id']
logo = LeaveOneGroupOut()

results = []
fold_predictions = {}

for fold_idx, (train_idx, test_idx) in tqdm(enumerate(logo.split(engineered_df, y, groups=groups)), desc="Iterating over folds", total=groups.nunique()):
    X_train_full, X_test_full = engineered_df.iloc[train_idx], engineered_df.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for model_name, model in models.items():
        for group_name, cols in feature_groups.items():
            X_train = X_train_full[cols]
            X_test = X_test_full[cols]

            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            # fallback for models without predict_proba
            if hasattr(model, "predict_proba"):
                y_proba = model.predict_proba(X_test)[:, 1]
            else:
                y_proba = model.predict(X_test)

            acc = accuracy_score(y_test, y_pred)
            try:
                auc = roc_auc_score(y_test, y_proba)
            except ValueError:
                # y_test might only contain one class
                auc = np.nan

            results.append({
                "fold": fold_idx,
                "model": model_name,
                "group": group_name,
                "accuracy": acc,
                "auc": auc
            })

            fold_predictions[(fold_idx, model_name, group_name)] = pd.DataFrame({
                "y_true": y_test.values,
                "y_pred": y_pred,
                "y_proba": y_proba
            }, index=y_test.index)

results_df = pd.DataFrame(results)

In [ ]:
# Aggregate by model and feature group
summary = results_df.groupby(["model", "group"])[["accuracy", "auc"]].agg(["mean", "std"]).reset_index()
# flatten multiindex
summary.columns = ["_".join(col).strip() for col in summary.columns.values]
summary['stability_score'] = summary['auc_mean'] - 0.5 * summary['auc_std']
summary.sort_values('stability_score', ascending=False, inplace=True)
display(summary)


***

# Winner

In [ ]:
best_hyperparams = summary.iloc[0]
best_model_name, best_feature_group = best_hyperparams["model"].iloc[0], best_hyperparams["group"].iloc[0]
print(f"Selected model: {best_model_name}; feature group: {best_feature_group}")

In [ ]:
best_model = models[best_model_name]
selected_cols = feature_groups.get(best_feature_group, feature_groups["All Features"])
selected_cols = [c for c in selected_cols if c in engineered_df.columns]

X_full = engineered_df[selected_cols]
y_full = y

best_model.fit(X_full, y_full)

In [ ]:
joblib.dump(best_model, FINAL_MODEL_PATH)
joblib.dump(selected_cols, FINAL_FEATURES_PATH)

***

# Results

## Top models

In [ ]:
top10 = summary.head(10)
plt.figure(figsize=(10,5))
sns.barplot(data=top10, x="group", y="stability_score", hue="model")
plt.xticks(rotation=45, ha="right")
plt.title("Top model+feature combinations by stability score")
plt.tight_layout()
plt.savefig(IMAGE_DIR / "top_models.png", dpi=RESOLUTION)
plt.close()

## Wilcoxon paired test

In [ ]:
from scipy.stats import wilcoxon

feature_group_to_compare = "All Features"

pairwise_tests = []
models_list = results_df["model"].unique()

for i, m1 in enumerate(models_list):
    for m2 in models_list[i+1:]:
        df1 = results_df[
            (results_df["model"] == m1) & 
            (results_df["group"] == feature_group_to_compare)
        ].sort_values("fold")

        df2 = results_df[
            (results_df["model"] == m2) & 
            (results_df["group"] == feature_group_to_compare)
        ].sort_values("fold")

        if len(df1) == len(df2) and len(df1) > 0:
            stat, p = wilcoxon(df1["auc"], df2["auc"])
            pairwise_tests.append({
                "model_1": m1,
                "model_2": m2,
                "wilcoxon_stat": stat,
                "p_value": p
            })

pd.DataFrame(pairwise_tests)

## Diagnostic plots

In [ ]:
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(6, 6))

for model_name in results_df["model"].unique():
    y_true_all = []
    y_proba_all = []
    
    for key, pred_df in fold_predictions.items():
        _, mname, group = key
        if mname == model_name and group == "All Features":
            y_true_all.extend(pred_df["y_true"])
            y_proba_all.extend(pred_df["y_proba"])

    fpr, tpr, _ = roc_curve(y_true_all, y_proba_all)
    roc_auc = auc(fpr, tpr)

    plt.plot(fpr, tpr, lw=2, label=f"{model_name} (AUC={roc_auc:.3f})")

plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Aggregated Across Folds)")
plt.legend()
plt.grid(True)
plt.savefig(IMAGE_DIR / "roc_curve.png", dpi=RESOLUTION)
plt.close()

In [ ]:
from sklearn.calibration import calibration_curve

plt.figure(figsize=(6, 6))

for model_name in results_df["model"].unique():
    y_true_all = []
    y_proba_all = []
    
    for key, pred_df in fold_predictions.items():
        _, mname, group = key
        if mname == model_name and group == "All Features":
            y_true_all.extend(pred_df["y_true"])
            y_proba_all.extend(pred_df["y_proba"])

    prob_true, prob_pred = calibration_curve(
        y_true_all, y_proba_all, n_bins=10, strategy="quantile"
    )

    plt.plot(prob_pred, prob_true, marker="o", label=model_name)

plt.plot([0, 1], [0, 1], "k--", label="Perfect")
plt.xlabel("Predicted Probability")
plt.ylabel("Empirical Win Rate")
plt.title("Calibration Plot")
plt.legend()
plt.grid(True)
plt.savefig(IMAGE_DIR / "calibration_curve.png", dpi=RESOLUTION)
plt.close()

In [ ]:
plt.figure(figsize=(8, 5))

for model_name in results_df["model"].unique():
    y_proba_all = []
    
    for key, pred_df in fold_predictions.items():
        _, mname, group = key
        if mname == model_name and group == "All Features":
            y_proba_all.extend(pred_df["y_proba"])

    sns.kdeplot(y_proba_all, label=model_name, fill=True, alpha=0.3)

plt.title("Distribution of Predicted Probabilities")
plt.xlabel("p(left wins)")
plt.legend()
plt.savefig(IMAGE_DIR / "predicted_probabilities_hist.png", dpi=RESOLUTION)
plt.close()


## Feature groups

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharex=False)

# --- Accuracy ---
sns.barplot(
    data=results_df,
    x="group",
    y="accuracy",
    errorbar="sd",
    ax=axes[0]
)
axes[0].set_title("Ablation Study: Accuracy by Feature Group")
axes[0].set_ylabel("Accuracy")
axes[0].set_xlabel("Feature Group")
axes[0].tick_params(axis='x', rotation=45)

# --- AUC ---
sns.barplot(
    data=results_df,
    x="group",
    y="auc",
    errorbar="sd",
    ax=axes[1]
)
axes[1].set_title("Ablation Study: AUC by Feature Group")
axes[1].set_ylabel("AUC")
axes[1].set_xlabel("Feature Group")
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(IMAGE_DIR / "feature_group_stats.png", dpi=RESOLUTION)
plt.close()


## Individual feature importance

In [ ]:
all_importances = []

for group_name, cols in feature_groups.items():
    X_full = engineered_df[cols]
    model = deepcopy(best_model)
    model.fit(X_full, y)
    
    df_imp = pd.DataFrame({
        "group": group_name,
        "feature": cols,
        "importance": model.feature_importances_
    })
    all_importances.append(df_imp)

importance_df = pd.concat(all_importances, ignore_index=True)

In [ ]:
# Pivot so rows=features, columns=groups, values=importance
heatmap_df = importance_df.pivot_table(
    index="feature", columns="group", values="importance", fill_value=0
)

plt.figure(figsize=(12,heatmap_df.shape[0]/5))
sns.heatmap(heatmap_df, cmap="viridis", linewidths=0.5)
plt.title("Feature Importance Heatmap Across Groups")
plt.xlabel("Feature Group")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()
plt.savefig(IMAGE_DIR / "feature_importance.png", dpi=RESOLUTION)

In [ ]:
global_ranking = importance_df.groupby("feature")["importance"].mean().sort_values(ascending=False).head(20)
plt.figure(figsize=(8,6))
sns.barplot(x=global_ranking.values, y=global_ranking.index, hue=global_ranking.index, palette="viridis")
plt.title("Top 20 Features Overall")
plt.xlabel("Mean Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()
plt.savefig(IMAGE_DIR / "top_20_features.png", dpi=RESOLUTION)

***

# Single feature models